<a href="https://colab.research.google.com/github/JainSahit/NLP576-SIA/blob/main/SahitJain/dcReading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers
!pip install semantic-text-similarity
!pip install urllib3==1.25.10
!pip install -U sentence-transformers

In [ ]:
import pyarrow
import numpy as np
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

In [ ]:
import transformers
import torch
import pickle
from tensorflow import keras
from datasets import list_datasets, load_dataset, list_metrics, load_metric
import json
from pprint import pprint
import pandas as pd
import pandas as pd
from tqdm import tqdm_notebook
from urllib3.util.url import IPV6_ADDRZ_RE
from semantic_text_similarity.models import WebBertSimilarity

In [ ]:
# Downloading and loading MS MARCO dataset
dataset = load_dataset('ms_marco', 'v2.1', split = 'validation[:10%]')

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/8378931e642240518368077ec1cc5b794130258f94ed47a957aba95e8910912a. Subsequent calls will reuse this data.


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.DataFrame.from_dict(dataset)
print(df.columns)
df.drop(['query_id', 'query_type', 'wellFormedAnswers'], axis=1)

In [ ]:
def preprocess(df):  
    count = 0
    count1 = 0
    queryList = []
    sentenceList = []
    label = []
    answerList = []

    for i, row in tqdm_notebook(df.iterrows()):
      #print("Question: ", df['query'][i])
      if df['wellFormedAnswers'][i] != []:
        #print("WellFormed Answer: ", df['wellFormedAnswers'][i])
        count1 += 1
      #print("Answer: ", df['answers'][i], "\n")
      if str(df['answers'][i]) == "['No Answer Present.']":
        count += 1
      
      if str(df['answers'][i]) != "['No Answer Present.']" or df['wellFormedAnswers'][i] != []:
          for idx, psg in enumerate(df['passages'][i]['passage_text']):
              if df['wellFormedAnswers'][i] != []:
                  answerList.append(str(df['wellFormedAnswers'][i]))
              else:
                  answerList.append(str(df['answers'][i]))
              sentenceList.append(psg)
              queryList.append(df['query'][i])
              label.append(np.nan)
      else:
          for idx, psg in enumerate(df['passages'][i]['passage_text']):
            queryList.append(df['query'][i])
            sentenceList.append(psg)
            answerList.append('')
            label.append(0)
    print("that was quite some work")
    qList = (queryList)
    sList = (sentenceList)
    labelList = (label)
    aList = (answerList)

    dfPair = pd.DataFrame({'Query': qList, 'Sentence': sList, 'Answer': aList, 'Label': labelList}) 
    print("Count of No Answers: ", count)
    print("Count of Well Formed Answers: ", count1)
    dfPair.sort_values(by=['Label'])
    print(dfPair[dfPair['Label'] == 0].count())
    dfPair['Query'] = dfPair['Query'].str.replace(".", '')
    for i in ['Sentence', 'Answer', 'Query']:
      dfPair[i] = dfPair[i].str.replace("(", '',)
      dfPair[i] = dfPair[i].str.replace(")", '',)
      dfPair[i] = dfPair[i].str.replace("[", '',)
      dfPair[i] = dfPair[i].str.replace("]", '',)
      dfPair[i] = dfPair[i].str.replace("'", '',)
    dfPair['Pair'] = dfPair['Query'] + ' ' + dfPair['Answer']
    return df

In [ ]:
dfPair = pd.read_csv('/content/drive/My Drive/pretraining1/df1/dfPair10K.csv')

In [ ]:
dfPair

,Unnamed: 0,Query,Sentence,Answer,Label,Pair
0,0,what is a corporation?,"A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly.",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
1,1,what is a corporation?,"Today, there is a growing community of more than 2,100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal: to redefine success in business. Join the Movement",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
2,2,what is a corporation?,"Corporation definition, an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of its members, and powers and liabilities distinct from those of its members. See more.",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
3,3,what is a corporation?,Examples of corporation in a Sentence. 1 He works as a consultant for several large corporations. 2 a substantial corporation that showed that he was a sucker for all-you-can-eat buffets.,A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
4,4,what is a corporation?,"1: a government-owned corporation as a utility or railroad engaged in a profit-making enterprise that may require the exercise of powers unique to government as eminent domain — called also government corporation, publicly held corporation",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
...,...,...,...,...,...,...
101047,101047,commercial insurance underwriter benefits,"Survey results imply that Underwriters deploy a deep pool of skills on the job. Most notably, facility with Financial Modeling, Commercial Loans, and Underwriting are correlated to pay that is significantly above average, leading to increases of 68 percent, 30 percent, and 15 percent, respectively.",NaN,0.0,commercial insurance underwriter benefits
101048,101048,commercial insurance underwriter benefits,Farmers Insurance Group Commercial Underwriter Reviews. 1 Work/life balance goes away during busy periods i in 64 reviews 2 Upper Management does not value the input of the Adjusters in 61 reviews 3 Long hours needed occasionally when work load is high in 40 reviews,NaN,0.0,commercial insurance underwriter benefits
101049,101049,commercial insurance underwriter benefits,"Commercial Lines - Insurance coverages for businesses, commercial institutions, and professional organizations, as contrasted with personal insurance. Commission - A portion of the policy premium that is paid to an agent by the insurance company as compensation for the agents work.",NaN,0.0,commercial insurance underwriter benefits
101050,101050,commercia

In [ ]:
dfPred[:1]

,Unnamed: 0,Query,Sentence,Answer,Label,Pair
0,0,what is a corporation?,"A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly.",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,NaN,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.


In [ ]:
dfZero[:1]

,Unnamed: 0,Query,Sentence,Answer,Label,Pair
20,20,why did the progressive movement fail to advance racial equality quizlet,"The Progressive Era spanned the years from 1890 to 1920 when the United States was experiencing rapid growth. Immigrants from eastern and southern Europe arrived in droves. Cities were overcrowded, and those living in poverty suffered greatly.",NaN,0.0,why did the progressive movement fail to advance racial equality quizlet


In [ ]:
mask = dfPair['Label'] != 0
dfZero = pd.DataFrame(dfPair[~mask])
dfPred = pd.DataFrame(dfPair[mask])

In [ ]:
# BERT imports
import torch
import torch.nn as nn
import pickle
from torch.utils.data import TensorDataset, DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
import transformers
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import time
import multiprocessing
from sentence_transformers import SentenceTransformer, util

% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
class Regressor(nn.Module):

  def __init__(self):
    super(Regressor, self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.out = nn.Linear(self.bert.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    output, pooler_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    score= self.out(pooler_out)
    return score
    
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=True)
customModel = torch.load('/content/drive/My Drive/pretraining1/model1/pytorch_modelTrained').to(device)

web_model = WebBertSimilarity(batch_size=4)
dBertBase = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens').to(device)
rBertLarge = SentenceTransformer('roberta-large-nli-stsb-mean-tokens').to(device)
rBertBase = SentenceTransformer('roberta-base-nli-stsb-mean-tokens').to(device)
bertBase = SentenceTransformer('bert-base-nli-stsb-mean-tokens').to(device)

In [ ]:
num_processes = multiprocessing.cpu_count()

def chunker(df):
  chunk_size = int(df.shape[0]/num_processes)
  return [df.loc[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]

def vectorize(d):
    tokenized = d.apply((lambda x: tokenizer.encode_plus(x.Pair, x.Sentence, add_special_tokens=True, max_length=128, 
                                                     return_token_type_ids=False, padding='max_length', 
                                                     return_attention_mask=True, truncation=True)), axis=1)
    return tokenized

def dataloader(d):
    ip_id = []
    masks = []

    for da in d:
      for i in da:
        ip_id.append(i['input_ids'])
        masks.append(i['attention_mask'])

    inputs = torch.tensor(ip_id).to(device)
    mask = torch.tensor(masks).to(device)
    data = TensorDataset(inputs, mask)
    dataloader = DataLoader(data, batch_size=4)

    return dataloader

def sts_score_generator(ten, model):
  scores = []
  for batch in tqdm_notebook(ten):
    batch = tuple(t.to(device) for t in batch)
    ip_ids,mask = batch
    score = model(ip_ids, attention_mask = mask)
    score = score.squeeze(1)
    scores.extend(score.detach().cpu().numpy())
  return scores

def PredictCustom(df):
  pool = multiprocessing.Pool(processes=num_processes)
  chunks = chunker(df)
  res = pool.map(vectorize, chunks)
  result = dataloader(res)
  return sts_score_generator(result, customModel)
  
def SentenceTransformerPreTrained(model, df):  
  scores = []
  for i, row in tqdm_notebook(df.iterrows()):
    embedding = model.encode([str(row['Pair']), str(row['Sentence'])], convert_to_tensor=True)
    score = util.pytorch_cos_sim(embedding[0], embedding[1])
    score = score.squeeze(1)
    scores.extend((4*score.detach().cpu().numpy()))
  return scores

def WebBERTPreTrained(df):
  scores = []
  for i, row in tqdm_notebook(df.iterrows()):
    scores.extend(web_model.predict([(str(row['Pair']), str(row['Sentence']))]))
  return scores

def SaveFixedLabel(df, name):
  df.loc[df.Label > 4, 'Label'] = 4
  df.loc[df.Label < 0, 'Label'] = 0
  df[['Query','Sentence','Label']].to_json('/content/drive/My Drive/pretraining1/result/' + name + '.json')


In [ ]:
dfPairCustom = dfPair
dfPairWbert = dfPair
dfPairDBert = dfPair
dfPairRlBert = dfPair
dfPairRbBert = dfPair
dfPairBert = dfPair
dfPredCustom = dfPred
dfPredWbert = dfPred
dfPredDBert = dfPred
dfPredRlBert = dfPred
dfPredRbBert = dfPred
dfPredBert = dfPred
dfZeroCustom = dfZero
dfZeroWbert = dfZero
dfZeroDBert = dfZero
dfZeroRlBert = dfZero
dfZeroRbBert = dfZero
dfZeroBert = dfZero

In [ ]:
dfPairCustom.Label = PredictCustom(dfPair)
dfPairWbert.Label = WebBERTPreTrained(dfPair)
dfPairDBert.Label = SentenceTransformerPreTrained(dBertBase, dfPair)
dfPairRlBert.Label = SentenceTransformerPreTrained(rBertLarge, dfPair)
dfPairRbBert.Label = SentenceTransformerPreTrained(rBertBase, dfPair)
dfPairBert.Label = SentenceTransformerPreTrained(bertBase, dfPair)
SaveFixedLabel(dfPairCustom, 'LabelledCustom')
SaveFixedLabel(dfPairWbert, 'LabelledWBert')
SaveFixedLabel(dfPairDBert, 'LabelledDBert')
SaveFixedLabel(dfPairRlBert, 'LabelledRlBert')
SaveFixedLabel(dfPairRbBert, 'LabelledRbBert')
SaveFixedLabel(dfPredBert, 'LabelledBert')

In [217]:
dfPairRbBert.Label = dfPairRbBert.Label.round(2)

In [218]:
dfPairRbBert

,Unnamed: 0,Query,Sentence,Answer,Label,Pair
0,0,what is a corporation?,"A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly.",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,2.70,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
1,1,what is a corporation?,"Today, there is a growing community of more than 2,100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal: to redefine success in business. Join the Movement",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,1.08,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
2,2,what is a corporation?,"Corporation definition, an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of its members, and powers and liabilities distinct from those of its members. See more.",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,3.27,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
3,3,what is a corporation?,Examples of corporation in a Sentence. 1 He works as a consultant for several large corporations. 2 a substantial corporation that showed that he was a sucker for all-you-can-eat buffets.,A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,1.43,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
4,4,what is a corporation?,"1: a government-owned corporation as a utility or railroad engaged in a profit-making enterprise that may require the exercise of powers unique to government as eminent domain — called also government corporation, publicly held corporation",A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.,2.24,what is a corporation? A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.
...,...,...,...,...,...,...
101047,101047,commercial insurance underwriter benefits,"Survey results imply that Underwriters deploy a deep pool of skills on the job. Most notably, facility with Financial Modeling, Commercial Loans, and Underwriting are correlated to pay that is significantly above average, leading to increases of 68 percent, 30 percent, and 15 percent, respectively.",NaN,1.56,commercial insurance underwriter benefits
101048,101048,commercial insurance underwriter benefits,Farmers Insurance Group Commercial Underwriter Reviews. 1 Work/life balance goes away during busy periods i in 64 reviews 2 Upper Management does not value the input of the Adjusters in 61 reviews 3 Long hours needed occasionally when work load is high in 40 reviews,NaN,0.83,commercial insurance underwriter benefits
101049,101049,commercial insurance underwriter benefits,"Commercial Lines - Insurance coverages for businesses, commercial institutions, and professional organizations, as contrasted with personal insurance. Commission - A portion of the policy premium that is paid to an agent by the insurance company as compensation for the agents work.",NaN,2.73,commercial insurance underwriter benefits
101050,101050,c